In [ ]:
using LinearAlgebra
using ForwardDiff
using SatelliteDynamics
using DifferentialEquations
using Plots
using ECOS, Convex
using Mosek, MosekTools

In [ ]:
#From RoadsMPC slide deck
#values are in m and degrees 
#orbits elements of und roads 1
#last is the true anomaly 
elements1 = [6885.635e3, 0.001089, 97.43, 0, 0, 0]

#orbital elements of und roads 2
#last is the true anomaly
elements2 = [6885.635e3, 0.001089, 97.43, 0, 0, 0.016667]

In [ ]:
#convert the true anomaly to mean anomaly to be able to do the conversion and degrees to radians

In [ ]:
function degrees_to_rad(x)

    return x*pi/180
    
end

In [ ]:
#v is true anomaly 
#e is the eccentricity
function true_anom_to_eccentric_anom(v, e)

    E = 2*atand(sqrt((1-e)/(1+e))*tand(v/2))

    #eccentric anomaly
    return E

end

In [ ]:
function eccentric_anom_to_mean_anom(E, e)

    M = E - e*sind(E)

    #mean anomaly
    return M

end

In [ ]:
#eccentric anomaly sat 1
E1 = true_anom_to_eccentric_anom(elements1[end], elements1[2])

#mean anomaly sat 1
M1 = eccentric_anom_to_mean_anom(E1, elements1[2])

In [ ]:
#eccentric anomaly sat 2
E2 = true_anom_to_eccentric_anom(elements2[end], elements2[2])

#eccentric anomaly sat 2
M2  = eccentric_anom_to_mean_anom(E2, elements2[2])

In [ ]:
#get all the units consistent (m, radians)

elements1_m = [elements1[1:2]; degrees_to_rad(elements1[3]); degrees_to_rad(elements1[4]); degrees_to_rad(elements1[5]); degrees_to_rad(M1)]

elements2_m = [elements2[1:2]; degrees_to_rad(elements2[3]); degrees_to_rad(elements2[4]); degrees_to_rad(elements2[5]); degrees_to_rad(M2)]

In [ ]:
#state satellite 1 (target)

x_1 = sOSCtoCART(elements1_m, use_degrees=false)

In [ ]:
#state satellite 2 (chaser)

x_2 = sOSCtoCART(elements2_m, use_degrees=false)

In [ ]:
#relative state of chaser wrt target
#this is in ECI frame
difference = x_2 - x_1 

In [ ]:
#this is in the RTN frame of the target (primary observing satellite) 
initial_difference = sECItoRTN(x_1, x_2) 

In [ ]:
#unit scaling to make the numerics better (no overshoot)
position_scale = 100
time_scale = 1000

velocity_scale = position_scale/time_scale
acceleration_scale = position_scale/(time_scale)^2

In [ ]:
#mu - gravitational parameter (m3/s2)
mu = 3.986e14

In [ ]:
mu_scaled = mu/(position_scale^3/time_scale^2)

In [ ]:
#target orbit period in seconds
T = 2*pi*sqrt((elements1[1]^3)/(mu))

In [ ]:
#semi major axis of target satellite (meters)
a = elements1[1]

In [ ]:
#scaled semi major axis
a_scaled = a/position_scale

In [ ]:
#constant mean motion of target n = sqrt(mu/a^3)
#units: rad/s
n = sqrt(mu/a^3)

In [ ]:
n_scaled = sqrt(mu_scaled/a_scaled^3)

In [ ]:
#number of states
nx = 6

#number of control inputs
nu = 3

In [ ]:
#timestep
dt = 60

In [ ]:
dt_scaled = dt/time_scale 

In [ ]:
#this is a 5 orbits for dt=60

#worked with 470 for some reason? 
#N = 470
#one orbit 
N = 94

#2 orbit look ahead? 
#N = 188

In [ ]:
thist = LinRange(1, N*dt, N)

In [ ]:
x0_target = zeros(nx) 

In [ ]:
#this is in RTN frame
# m and s
x0_chaser = initial_difference

In [ ]:
A = zeros(nx, nx)
A[1:3, 4:6] = I(3)
A[4, 1] = 3*n^2
A[6, 3] = -n^2
A[4, 5] = 2*n
A[5, 4] = -2*n

In [ ]:
A_scaled = zeros(nx, nx)
A_scaled[1:3, 4:6] = I(3)
A_scaled[4, 1] = 3*n_scaled^2
A_scaled[6, 3] = -n_scaled^2
A_scaled[4, 5] = 2*n_scaled
A_scaled[5, 4] = -2*n_scaled

In [ ]:
#mass of the satellite (kg) from data
m = 5.22

In [ ]:
#in m/s2. L1 norm 
umax = 4.6e-3

In [ ]:
umax_scaled = umax/acceleration_scale 

In [ ]:
#si units
B = [zeros(3,3); I(3)]

In [ ]:
#continous dynamics
function spacecraft_dynamics(x,u)

    xdot = A*x + B*u

    return xdot

end

In [ ]:
#discretize the dynamics model 
H_scaled = exp(dt_scaled*[A_scaled B; zeros(nu, nx+nu)])

In [ ]:
#Scaled Discrete Dynamics Matrices
Ad_scaled  = H_scaled[1:nx, 1:nx]
Bd_scaled = H_scaled[1:nx, (nx+1):end]

In [ ]:
#discretize the dynamics model 
H = exp(dt*[A B; zeros(nu, nx+nu)])

In [ ]:
#Discrete Dynamics Matrices
Ad  = H[1:nx, 1:nx]
Bd = H[1:nx, (nx+1):end]

In [ ]:
#discrete dynamics
function spacecraft_dynamics_discrete(x,u)

    xnext = Ad*x + Bd*u

    return xnext

end

In [ ]:
function spacecraft_dynamics_discrete_scaled(x_scaled, u_scaled)

    xnext_scaled = Ad_scaled*x_scaled + Bd_scaled*u_scaled

    return xnext_scaled

end

In [ ]:
N_h = N

In [ ]:
x_initial = x0_chaser

In [ ]:
x_initial_scaled = [x0_chaser[1:3]/position_scale; x0_chaser[4:6]/velocity_scale] 

In [ ]:
x_goal = x0_target

In [ ]:
function update_prob(x_initial_k)


    X = Convex.Variable(nx, N_h)
    U = Convex.Variable(nu, N_h-1)
    
    #Initial State Constraint
    cons = Constraint[X[:,1] == x_initial_k]

    #Dynamics Constraint
    for k=1:(N_h-1)

        push!(cons, X[:,k+1] - (Ad_scaled*X[:,k] + Bd_scaled*U[:,k]) == zeros(6))

    end

    #add in thrust constraints

    #try both l1 and l2
    for k=1:(N_h-1)

       #L2 thrust constraint
       #push!(cons, norm(U[:,k],2) <= umax/acceleration_scale)

    #    #L1 thrust constraint
    #works when bound is 1
       push!(cons, norm(U[:,k],1) <= (umax/60/acceleration_scale))


       #bound constraints (working)
    #    push!(cons, U[1,k] <= 4.6e-3)
    #    push!(cons, U[2,k] <= 4.6e-3)
    #    push!(cons, U[3,k] <= 4.6e-3)
    #    push!(cons, U[1,k] >= -4.6e-3)
    #    push!(cons, U[2,k] >= -4.6e-3)
    #    push!(cons, U[3,k] >= -4.6e-3)

    end


    #60 seconds between each control constraint 
    # for k=1:(N_h-1)

    #     if (k%2) == 0

    #         push!(cons, U[:,k] == zeros(3))

    #     else
    #         continue
    #     end


    # end



    return cons, X, U

end

In [ ]:
function solve_prob(cons, X, U, N)


    #objective 
    obj = 0

    #weights depending how conservative you want to be on fuel vs reaching the goal
    #sensative to these weights. with beta as 100 it wouldn't solve...
    beta = 1
    alpha = 1

    for k=1:N-1
        obj += beta*norm(U[:,k], 1) + alpha*norm(X[:,k], 2)
    end

    #terminal cost
    obj += alpha*norm(X[:,N], 2)

    prob = minimize(obj, cons)

    #solve with ECOS
    #Convex.solve!(prob, ()-> ECOS.Optimizer(), silent_solver=true);

    Convex.solve!(prob, ()-> Mosek.Optimizer(), silent_solver = true)

    Xm = X.value
    Um = U.value

    return Xm, Um

end

In [ ]:
#iterations = 1000
num_orbits = 3

iterations = N*num_orbits

In [ ]:
#set up in Convex.jl

In [ ]:
xhist = zeros(nx, iterations)
uhist = zeros(nu, iterations-1)

xhist[:,1] = x_initial_scaled

In [ ]:
#for 60 seconds constraint
iters_odd = 1:2:iterations-2

In [ ]:
for i=1:iterations-1

#test
#for i=1

#for 60 second constraint
#for i in iters_odd 
    cons, X, U = update_prob(xhist[:,i])

    Xm, Um = solve_prob(cons, X, U, N_h)

    #take one step 
    uhist[:,i] = Um[:,1]

    #uhist[:,i+1] = Um[:,2]

    xhist[:,i+1] = spacecraft_dynamics_discrete_scaled(xhist[:,i], uhist[:,i])

    #xhist[:,i+2] = spacecraft_dynamics_discrete(xhist[:,i+1], uhist[:,i+1])

end

In [ ]:
plot(Xm')

In [ ]:
xhist_si = [xhist[1:3, :]*position_scale; xhist[4:6, :]*velocity_scale]  

In [ ]:
#rescale uhist into m/s2

uhist_si = uhist*acceleration_scale 

In [ ]:
plot(xhist_si[1,:], xhist_si[2,:], xhist_si[3,:], label = "Chaser Trajectory", zlim = (-1,1))

scatter!([xhist_si[1,1]], [xhist_si[2,1]], [xhist_si[3,1]], label = "Initial State")
scatter!([xhist_si[1,end]], [xhist_si[2,end]], [xhist_si[3,end]], label = "End State")

scatter!([0], [0], [0], label= "Target")

In [ ]:
plot(xhist_si[2,:], xhist_si[1,:], xlabel= "Tangential (m)", ylabel="Radial (m)")

In [ ]:
plot(xhist_si[3,:], xhist_si[1,:], xlabel= "Normal (m)", ylabel="Radial (m)", label=false)

In [ ]:
plot(xhist_si[1,:], title="Positions") 
plot!(xhist_si[2,:])
plot!(xhist_si[3,:])

In [ ]:
plot(xhist_si[4,:], title="Velocities") 
plot!(xhist_si[5,:])
plot!(xhist_si[6,:])

In [ ]:
plot(uhist_si[1,:], title = "Controls")
plot!(uhist_si[2,:])
plot!(uhist_si[3,:])

In [ ]:
fuel_usage_accel = sum(abs.(vec(uhist_si)))

In [ ]:
delta_v = dt*fuel_usage_accel 

In [ ]:
no_control = zeros(nx, iterations)

In [ ]:
no_control[:,1] = x0_chaser

In [ ]:
for i = 1:iterations-1

    no_control[:, i+1] = Ad*no_control[:,i]

end

In [ ]:
# plot(no_control[1,:], no_control[2,:], no_control[3,:], zlims=(-1,1))
# scatter!([0], [0], [0])

In [ ]:
plot(no_control[2,:], no_control[1,:])

In [ ]:
plot(no_control[3,:], no_control[1,:])